In [10]:
import numpy as np
import math
import random
import matplotlib.pyplot as plt
import time

In [2]:
from keras.models import Sequential
from keras.layers import LSTM, Flatten, Dense, Dropout

In [3]:
# Carregando Dataset
def dataset():
    import pandas_datareader.data as web
    import datetime as dt

    end = dt.datetime(2020, 11, 1)
    start = dt.datetime(2015, 1, 1)

    df = web.DataReader("AAPL", 'yahoo', start, end)

    df = df.reset_index()
    df = df.drop(columns=['Open','Date','High','Low','Volume','Adj Close'])
    df = df.rename(columns={'Close': 'Close 0'})

    def window (df, w):
        for i in range(1,w):
            df['Close '+str(i)] = df['Close '+str(i-1)].shift(1)
        return df

    new_df = window(df,5)
    new_df = new_df.rename(columns={'Close 0': 'Target'})
    new_df.dropna(inplace=True)

    limit = int(len(new_df)*0.75)

    X = new_df.loc[:,['Close 1','Close 2','Close 3','Close 4']]
    y = new_df.loc[:,'Target'].tolist()

    X = np.array(X)
    X = X.reshape(X.shape[0], X.shape[1], 1)

    y = np.array(y)

    X_train, X_test = X[:limit], X[limit:]
    y_train, y_test = y[:limit], y[limit:]
    
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = dataset()

### Definindo hiperparâmetros

In [4]:
# batch_size e epochs
batch_size = [8, 16, 32, 64, 128]
epochs = [50, 100, 200, 500]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
dropout_rate = [0.0, 0.1, 0.2, 0.3]
node_first = [30, 50, 75]
node_last = [5, 7, 10]
hidden = [2,3,4]

param_grid = dict(batch_size = batch_size, epochs = epochs, optimizer = optimizer,
                    activation = activation, dropout_rate = dropout_rate,
                    node_first = node_first, node_last = node_last, hidden = hidden)

### Classe genética

In [15]:
class genetic:
    
    def __init__(self, n_geracoes=10, populacao=10, space = {}):
        print("Created")
        
    #Definindo população inicial
    def set_pop(self, size):
        
        individuos = []
    
        for p in range(size):
            cromossomo = []
            for gene in self.space:
                cromossomo.append(random.choice(gene))

            individuos.append(cromossomo)

        print("População ok")
        return individuos
    
    #Definindo tipos
    def types_genes(self):
        #Armazenando tipos
        self.types = [type(g[0]) for g in self.space]
        print("Genes tipificados")
        
    #Definindo objetivo
    def obj(self, cr, idx = 0):
        return 0
        
    
    #Ordenar pontos e cromossomos
    def ordena (self, pontos, cromo):
        zipped = zip(pontos, cromo)

        ordenados = sorted(zipped)

        pontos_ord = []
        cromo_ord = []
        for p, c in ordenados:

            pontos_ord.append(p)
            cromo_ord.append(c)

        print("Ordenação ok")
        return pontos_ord, cromo_ord
    
    #Mutações
    def mutacao(self, gf, gene):
        #gerando taxa de mutação utilizando gaussiana de desvio 0.1
        rate = np.random.normal(1, 0.1)
        
        if self.types[gene] is type(0): #int
            gf = int(gf * rate)
            if gf > self.max_val[self.dict_[gene]]:
                gf = self.max_val[self.dict_[gene]]
                
            elif gf < self.min_val[self.dict_[gene]]:
                gf = self.min_val[self.dict_[gene]]

        elif self.types[gene] is type(0.0): #float
            gf = round(gf * rate, 3)
            if gf > self.max_val[self.dict_[gene]]:
                gf = self.max_val[self.dict_[gene]]
                
            elif gf < self.min_val[self.dict_[gene]]:
                gf = self.min_val[self.dict_[gene]]

        elif self.types[gene] is type('0'): #string
            gf = gf
        
        print("Xmen mutados")
        return gf
    
    #Fazendo cruzamentos
    def crossover(self, cromo_, pesos):
        nova_geracao = []
        for c in range (self.pop):
            pai = random.choices(cromo_, weights=pesos)[0]
            id_pai = cromo_.index(pai)

            #Se houver mais de 1 cromossomo para a reprodução os pais devem ser diferentes
            if len(cromo_) > 1:
                cromo_.pop(id_pai)
                pesos.pop(id_pai)

            mae = random.choices(cromo_, weights=pesos)[0]

            filho = []
            for gene in range(len(pai)):
                gp = pai[gene]
                gm = mae[gene]
                gf = random.choice([gp, gm])
                
                #Adicionando mutação
                gf = self.mutacao(gf, gene)

                filho.append(gf)

            nova_geracao.append(filho)

        print("Nascimento de uma nova geração")
        return nova_geracao
    
    #Função de parada - Se a maior pontuação e o melhor cromossomo não alterararem por 5 gerações seguidas
    def stop(self, cromos, pontos):
        
        equal = 0
        
        if len(self.top_cromos) < 5:
            self.top_pontos.append(pontos[-1])
            self.top_cromos.append(cromos[-1])
            
        else:
            #verificar se o maior ponto é maior que o top
            if pontos[-1] > min(self.top_pontos):
                #apagar a menor pontuação
                minimo = min(self.top_pontos)
                min_id = self.top_pontos.index(minimo)
                self.top_pontos.pop(min_id)
                self.top_cromos.pop(min_id)
                
                #adicionar cromossomo e pontuações atuais
                self.top_pontos.append(pontos[-1])
                self.top_cromos.append(cromos[-1])
            
            #Verificar se os três melhores cromossomos são iguais
            #Se forem iguais significa que não está atualizando, ou seja, pode parar
            for cr in self.top_cromos:
                if cr  == self.top_cromos [0]:
                    equal += 1
                    
        #Todos valores iguais
        if equal == 5:
            return True
        
        return False
                
        
    
    #Função da evolução
    def evolucao(self):
        print("entrou na evolucao")
        self.types_genes()
        self.cromo = self.set_pop(self.pop)
        self.parar = False
        for ger in range(self.geracoes):
            
            if ger == 0:
                self.pontos = self.obj(self.cromo, 0)
                print("Pontos: ",self.pontos)
                self.pontos_max = max(self.pontos)
                print("MAX: ",self.pontos_max)
                self.id_max = self.pontos.index(self.pontos_max)
                self.vencedor = self.cromo[self.id_max]
            
            else:
                self.cromo = self.cromo_ord
                self.pontos = self.pontos_ord
            
            #Calculando pontuação
            print("GERAÇÃO: ",ger)
            print("Cromossomos: ",self.cromo)
            print("Pontuações: ", self.pontos)

            self.final_pont_max.append(max(self.pontos))

            #Atualizando cromossomo vencedor
            if(max(self.pontos) > self.pontos_max):
                self.pontos_max = max(self.pontos)
                self.id_max = self.pontos.index(self.pontos_max)
                self.vencedor = self.cromo[self.id_max]

            #Organizar listas de acordo com a pontuação
            self.pontos_ord, self.cromo_ord = self.ordena(self.pontos, self.cromo)
                
            #Guardando 20% dos melhores para, além de serem pais, serem utilizados na nova geração
            self.master = self.cromo_ord[int(math.floor(len(self.cromo_ord)*0.8)):]

            #print(taxa)
            #self.ponto_cut, self.cromo_cut = self.limiar(self.pontos_ord, self.cromo_ord)
            self.ponto_cut, self.cromo_cut = self.pontos_ord, self.cromo_ord
            
            #Definindo pesos para os cromossomos
            self.pesos = [i/self.max_pontos for i in self.pontos_ord]

            #Crossover
            self.geracao = self.crossover(self.cromo_ord, self.pesos)
            
            #Adicionando alguns intrusos
            intrusos = self.set_pop(len(self.geracao)*0.2)
            for i in intrusos:
                self.geracao.append(i)

            #Adicionando melhores da geração anterior
            for m in self.master:
                self.geracao.append(m)
                
            #deletando os 20% piores
            self.pontos = self.obj(self.geracao, 0)
            self.pontos_ord, self.cromo_ord = self.ordena(self.pontos, self.geracao)
            self.cromo_ord = self.cromo_ord[int(math.floor(len(self.cromo_ord)*0.2)):]
            self.pontos_ord = self.pontos_ord[int(math.floor(len(self.pontos_ord)*0.2)):]
            
            

            print("--------")
            ##self.parar = self.stop(self.cromo_ord, self.pontos_ord)
            self.parar = 0
            if self.parar:
                #Calculando pontuação
                print("FINAL: ")
                print("Cromossomos: ",self.cromo_ord)
                print("Pontuações: ", self.pontos_ord)

                self.final_pont_max.append(max(self.pontos_ord))

                #Atualizando cromossomo vencedor
                self.pontos_max = max(self.pontos_ord)
                self.id_max = self.pontos.index(self.pontos_max)
                self.vencedor = self.cromo_ord[self.id_max]
                break;

        print("Melhor pontuação: ", self.pontos_max)
        self.best_config = dict(zip(self.dict_, self.vencedor))
        print("Melhor cromossomo: ", self.best_config)
        print("Numero de gerações: ", (ger+1))
        
        #retornando modelo
        return self.obj(self.vencedor, 1)

In [26]:
class geneticNN(genetic):
    
    def __init__(self, n_geracoes=10, populacao=10, space = {}):
        
        self.geracoes = n_geracoes
        self.pop = populacao
        self.cromo = []
        self.pontos = []
        self.final_pont_max = []
        self.pontos_max = 0
        self.max_pontos = 100
        self.id_max = 0
        self.vencedor = []
        self.dict_ = list(space)
        self.space = list(space.values())
        self.types = []
        self.top_pontos = []
        self.top_cromos = []
        self.max_val = dict(zip(space.keys(),[max(space[s]) for s in space]))
        self.min_val = dict(zip(space.keys(),[min(space[s]) for s in space]))
        
        print("Genetic NN OK")
        
    def obj(self, cr, idx = 0): #idx = 0 retorna o resultado, idx = 1 retorna o modelo
        result = []
        size = len(cr[0]) #quantidade de genes
        for cromos in cr:
    
            self.batch_size = cromos[self.dict_.index("batch_size")]
            self.epochs = cromos[self.dict_.index('epochs')]
            self.optimizer = cromos[self.dict_.index('optimizer')]
            self.activation = cromos[self.dict_.index("activation")]
            self.dropout_rate = cromos[self.dict_.index("dropout_rate")]
            self.node_first = cromos[self.dict_.index("node_first")]
            self.node_last = cromos[self.dict_.index("node_last")]
            self.hidden = cromos[self.dict_.index("hidden")]

            modelo = self.create_model()
            
            modelo.fit(X_train, y_train, epochs = self.epochs, batch_size = self.batch_size, verbose = 0)
            
            res = modelo.evaluate(X_test, y_test)
            
            res = res * -1 #Transformando em negativo, para diminuir o erro

            #print(res)
            res = round(res, 3)
            if math.isnan(res):
                res = -math.inf
                
            result.append(res)

        if idx == 0:
            return result
        return modelo
    
    #Função para selecionar nós
    def find_nodes(self):
        layers = []
        if self.hidden == 1:
            increment = (self.node_first + self.node_last) / 2
        else:
            increment = (self.node_last - self.node_first) / (self.hidden)


        #primeiro nó
        layers.append(self.node_first)

        nodes = self.node_first

        for i in range (0, self.hidden):
            nodes = nodes + increment
            layers.append(math.ceil(nodes))

        return layers

    #Criando modelo base
    def create_model(self):

        model = Sequential()
        model.add(Flatten())
        nodes = self.find_nodes()
        
        for i in range(0, self.hidden+1):
            if i == 0:
                model.add(Dense(self.node_first, activation = self.activation))
            else:
                model.add(Dense(nodes[i], activation = self.activation))
                model.add(Dropout(self.dropout_rate))

        model.add(Dense(1))

        model.compile(loss='mse', optimizer=self.optimizer)

        return model
    

In [23]:
gnn = geneticNN(n_geracoes = 5, populacao = 10, space = param_grid)

Genetic NN OK


In [24]:
ini = time.time()
gnn.evolucao()
fim = time.time()

entrou na evolucao
Genes tipificados
População ok
12/12 [==============================] - 0s 2ms/step - loss: 810.3356
Pontos:  [-5040.83, -5.206, -2589.609, -916.211, -2095.059, -5987.469, -102.954, -4.595, -854.311, -810.336]
MAX:  -4.595
GERAÇÃO:  0
Cromossomos:  [[64, 200, 'Nadam', 'softmax', 0.2, 50, 10, 4], [32, 50, 'Nadam', 'softplus', 0.1, 50, 5, 3], [16, 50, 'Adamax', 'sigmoid', 0.1, 30, 5, 2], [64, 500, 'Adam', 'softsign', 0.0, 75, 7, 3], [8, 200, 'Adamax', 'hard_sigmoid', 0.1, 50, 7, 2], [128, 500, 'Adadelta', 'sigmoid', 0.0, 30, 5, 2], [32, 100, 'Adam', 'relu', 0.1, 75, 5, 3], [16, 200, 'RMSprop', 'linear', 0.0, 30, 10, 3], [32, 500, 'RMSprop', 'sigmoid', 0.2, 50, 7, 2], [16, 200, 'Nadam', 'sigmoid', 0.0, 75, 7, 3]]
Pontuações:  [-5040.83, -5.206, -2589.609, -916.211, -2095.059, -5987.469, -102.954, -4.595, -854.311, -810.336]
Ordenação ok
Xmen mutados
Xmen mutados
Xmen mutados
Xmen mutados
Xmen mutados
Xmen mutados
Xmen mutados
Xmen mutados
Xmen mutados
Xmen mutados
Xmen 

12/12 [==============================] - 0s 2ms/step - loss: 12.1869
Ordenação ok
--------
GERAÇÃO:  2
Cromossomos:  [[16, 50, 'Nadam', 'hard_sigmoid', 0.1, 50, 7, 3], [16, 200, 'SGD', 'softsign', 0.2, 75, 10, 3], [128, 50, 'SGD', 'tanh', 0.0, 30, 7, 2], [33, 447, 'Adam', 'sigmoid', 0.0, 45, 5, 4], [8, 200, 'Adamax', 'tanh', 0.0, 75, 5, 3], [31, 398, 'Adam', 'sigmoid', 0.0, 46, 5, 2], [32, 500, 'RMSprop', 'softplus', 0.3, 30, 7, 3], [35, 304, 'Nadam', 'softplus', 0.108, 72, 5, 3], [39, 50, 'Adam', 'softplus', 0.084, 75, 5, 3], [31, 187, 'Nadam', 'softplus', 0.0, 35, 9, 2], [16, 200, 'RMSprop', 'linear', 0.0, 30, 10, 3], [8, 200, 'Adam', 'linear', 0.2, 50, 5, 3], [41, 500, 'Adagrad', 'softplus', 0.081, 68, 10, 2], [67, 209, 'Adagrad', 'softplus', 0.0, 37, 7, 2], [20, 242, 'RMSprop', 'linear', 0.0, 30, 6, 3], [37, 54, 'Nadam', 'softplus', 0.081, 45, 5, 2], [16, 50, 'Adamax', 'relu', 0.0, 75, 10, 4], [64, 200, 'RMSprop', 'softplus', 0.0, 30, 7, 2], [31, 181, 'Adam', 'softplus', 0.0, 71, 5

12/12 [==============================] - 0s 2ms/step - loss: 5.8162
Ordenação ok
--------
GERAÇÃO:  4
Cromossomos:  [[16, 500, 'Adagrad', 'relu', 0.1, 50, 7, 4], [16, 200, 'RMSprop', 'relu', 0.2, 30, 7, 2], [8, 100, 'Adadelta', 'softplus', 0.2, 50, 5, 2], [32, 500, 'Adadelta', 'softplus', 0.2, 50, 10, 2], [49, 56, 'Adamax', 'softplus', 0.0, 34, 10, 2], [35, 231, 'RMSprop', 'relu', 0.0, 43, 7, 2], [43, 191, 'Adagrad', 'softplus', 0.0, 71, 10, 2], [16, 228, 'RMSprop', 'softplus', 0.0, 30, 7, 2], [31, 166, 'Adagrad', 'softplus', 0.0, 75, 5, 2], [128, 100, 'Nadam', 'softplus', 0.0, 75, 5, 4], [46, 50, 'Adamax', 'relu', 0.0, 75, 9, 3], [17, 180, 'Nadam', 'softplus', 0.0, 69, 5, 2], [27, 203, 'RMSprop', 'softplus', 0.0, 43, 9, 2], [31, 181, 'Adam', 'softplus', 0.0, 71, 5, 3], [16, 52, 'Nadam', 'softplus', 0.0, 33, 5, 2], [17, 53, 'Adamax', 'linear', 0.0, 75, 10, 4], [16, 200, 'Adamax', 'softplus', 0.0, 30, 10, 3], [35, 234, 'Nadam', 'softplus', 0.0, 31, 5, 3], [28, 191, 'Nadam', 'softplus', 

In [25]:
print((fim - ini)/60)

38.583046146233876


In [47]:
help(math)

Help on built-in module math:

NAME
    math

DESCRIPTION
    This module provides access to the mathematical functions
    defined by the C standard.

FUNCTIONS
    acos(x, /)
        Return the arc cosine (measured in radians) of x.
    
    acosh(x, /)
        Return the inverse hyperbolic cosine of x.
    
    asin(x, /)
        Return the arc sine (measured in radians) of x.
    
    asinh(x, /)
        Return the inverse hyperbolic sine of x.
    
    atan(x, /)
        Return the arc tangent (measured in radians) of x.
    
    atan2(y, x, /)
        Return the arc tangent (measured in radians) of y/x.
        
        Unlike atan(y/x), the signs of both x and y are considered.
    
    atanh(x, /)
        Return the inverse hyperbolic tangent of x.
    
    ceil(x, /)
        Return the ceiling of x as an Integral.
        
        This is the smallest integer >= x.
    
    copysign(x, y, /)
        Return a float with the magnitude (absolute value) of x but the sign of y.
   

In [46]:
lst = ['aa','bb','cc','dd']

r = math.rand()
print(r)

AttributeError: module 'math' has no attribute 'rand'